# (2) Implement the ANN algorithm.

In [28]:
from math import exp
from random import random

def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

def transfer_derivative(output):
    return output * (1.0 - output)

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
    
dataset = [[0.6, 0.1, 0], [0.3, 0.2, 1]]
network = [[{'weights': [0.1, -0.2]}, {'weights': [0, 0.2]}, {'weights': [0.3, -0.4]}],
           [{'weights': [-0.4, 0.1, 0.6]}, {'weights': [0.2, -0.1, -0.2]}]]
for row in dataset:
    prediction = predict(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))
    
network = initialize_network(2, 3, 2)
train_network(network, dataset, 0.1, 1, 2)
for layer in network:
    print(layer)

Expected=0, Got=0
Expected=1, Got=0
[{'weights': [0.7100239977021047, 0.9360845885016724, 0.41962281453193007], 'output': 0.6944618503730177, 'delta': -0.010757975483869676}, {'weights': [0.8292956843949971, 0.6699370390277636, 0.3013178542237483], 'output': 0.6650633934192615, 'delta': -0.016346171539056195}, {'weights': [0.5873835466907534, 0.8824437121317008, 0.8458608344802276], 'output': 0.768272622448061, 'delta': -0.00016303053483261045}]
[{'weights': [0.4989407856304778, 0.5830039699381204, 0.027446563989363446, 0.23342568742554343], 'output': 0.7358891014189113, 'delta': -0.1430247063867633}, {'weights': [0.7907747639408322, 0.40778308911053335, 0.16582153918324105, 0.5396837697871159], 'output': 0.8146337585847377, 'delta': 0.027991340126294548}]
